In [ ]:
%%capture
import os
import numpy as np
import mne
import csv
import re
from mne.channels import make_standard_montage
from glob import glob
from mne_bids import BIDSPath,  write_raw_bids
from pyprep.prep_pipeline import PrepPipeline #https://pyprep.readthedocs.io/en/latest/index.html 
from pyprep.find_noisy_channels import NoisyChannels

root_source ='C:/Users/mfbpe/Desktop/DATA/2022_deliberation/raw'
root_derivatives = root_source.split('/raw')[0] + '/derivatives'
root_image =root_source.split('/raw')[0] + '/images_prep_eeg/'

os.chdir(root_derivatives)

# all_files_path = sorted(glob(f'{root_source}/*/*/*.bdf'), key=len)
all_files_path = sorted(glob(f'*sampled_raw.fif'), key=len)

list_bad = {'001': ['Pz'], '002': ['T8'], '003': ['T7'], '004': ['PO3', 'O2', 'P3', 'Oz'], '005': ['Pz', 'O1', 'Oz'], 
'006': ['Pz', 'PO3'], '007': ['T8'], '008': ['T8'], '009': ['Pz'], '010': ['CP5', 'T7'], '011': [], '012': ['Pz', 'T8'], 
'013': ['Pz'], '014': [], '015': ['O1'], '016': ['PO4', 'PO3'], '017': ['PO3', 'F4'], '018': ['PO3'], 
'019': ['F7', 'O2', 'F3', 'O1', 'P7', 'Oz'], '020': [], '021': ['Pz', 'PO3'], '022': ['PO4', 'PO3', 'T7', 'Oz'], 
'023': ['Pz', 'PO3'], '024': ['PO3', 'T7'], '025': ['Pz', 'P3'], '026': ['F3', 'O1'], '027': ['Pz', 'Oz', 'O1', 'PO3'], 
'028': ['O2', 'Pz', 'O1', 'PO3', 'Oz'], '029': ['O1', 'PO3', 'PO4', 'T7'], '030': ['Pz', 'O1', 'PO3'], '031': ['PO3', 'T7', 'Fp1'],
 '032': ['PO3', 'Pz', 'O1'], '033': ['P4', 'Fp2', 'Fp1', 'O1', 'P7', 'PO3', 'T7', 'P3'], '034': ['T8', 'T7'],
  '035': ['CP2', 'Pz', 'C4', 'P4'], '036': ['PO3', 'T8', 'T7'], '037': [], '038': ['PO3', 'P4', 'P3'], '039': [], '040': [],
   '041': ['F7', 'P3'], '042': ['T8', 'T7'], '043': ['P7'], '044': [], '045': ['PO4', 'F7', 'PO3', 'Fp1'], '046': ['PO3'], 
   '047': ['O1', 'T8', 'T7'], '048': ['P7'], '049': ['PO3', 'P3'], '050': ['PO3']}
list_ica = {'001':[1,2],'050':[0,6],'006':[1,3],'007':[2,8],'008':[2,10],
'012':[0,2],'013':[0,6],'015':[0,9],'023':[0,4],'024':[3,7],
'029':[0,3],'031':[4,10],'032':[1,4],'040':[1,3],'041':[0,3]}

for part in range(len(all_files_path)):
    
    n_part = re.split('sub-|_2023',all_files_path[part])[1].rjust(3,'0') # to have the participant number in bids format e.g '001'

    raw = mne.io.read_raw_fif(all_files_path[part])
    
    
    # ##############################################################################
    # ############                  ################################################
    # ############  Pre-processing  ################################################
    # ############                  ################################################
    # ##############################################################################
    raw.load_data()
    events = mne.find_events(raw, shortest_event=1)
    try:
        raw.crop(((events[1,0]/512))-5, (events[-1,0]/512)+5)
    except : 
        pass

    raw.set_channel_types({'EXG1':'eeg','EXG2':'eeg','EXG3': 'eeg', 'EXG4':'eeg'})

    raw.filter(l_freq=.1,h_freq=40, n_jobs=-1) #filter bandpass 0.1 - 40 Hz


    ## Use a copy to perform the ICA on harshly high pass
    r=raw.copy() 
    raw.set_channel_types({'EXG1':'misc','EXG2':'misc','EXG3': 'eog', 'EXG4':'eog'})

    r.filter(l_freq=1,h_freq=None, n_jobs=-1)
    r.set_channel_types({'EXG1':'misc','EXG2':'misc','EXG3': 'eog', 'EXG4':'eog'})

    # Interpolate bad channels for participants in the prtp_chs_out list
    if n_part in list(list_bad):
            
        raw.info['bads'] = list_bad[n_part]
        r.info['bads'] = list_bad[n_part]
    else :
        noisy= NoisyChannels(r)
        noisy.find_all_bads()
        bad_chs = noisy.get_bads()
        bad_chs = [x for x in bad_chs if x  not in ['Fz','Cz']]
        raw.info['bads'] = bad_chs
        r.info['bads'] = bad_chs
        list_bad[n_part]=bad_chs


    raw = raw.interpolate_bads()
    r = r.interpolate_bads()


    ##########################################################################
    ############       #######################################################
    ############  ICA  #######################################################
    ############       #######################################################
    ##########################################################################

    ica = mne.preprocessing.ICA(random_state=21, max_iter='auto')
    ica.fit(r)
    
    ica.exclude = []
    # find which ICs match the EOG pattern
    r.set_channel_types({'Fp1': 'eog', 'Fp2': 'eog'})

    if n_part in list_ica:
        ica.exclude = list_ica[n_part]
    else : 
        eog_indices, eog_scores = ica.find_bads_eog(r,measure= 'correlation', threshold = .5)
        ica.exclude = eog_indices
    del r
    if len(ica.exclude) >0:
        # plot diagnostics
        ica.apply(raw)

        fig = ica.plot_components(picks=np.arange(ica.n_components_), show = False) #create figures of the components with the one removed
        fig.savefig(f"{root_image}sub-{n_part}_ICA.png")
    del ica
    raw.save(f"sub-{n_part}_2023_ica_raw.fif",overwrite=True)

    del raw,r


        

# create the txt file

with open('list_bad_channels.txt', 'w') as f:
    f.write('dict = ' + repr(list_bad) + '\n')  

with open('list_ica.txt', 'w') as f:
    f.write('dict = ' + repr(list_bad) + '\n')  
